In [93]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool

import os
import requests

In [94]:
# configure tools 
pse_key = os.getenv('API_KEY')

@tool
def google_search(query: str):
    """Perform Google Custom Search and return top result snippet."""
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": pse_key,
        "cx": '84da54018bf7d48ed',
        "q": query,
        "num": 1
    }
    response = requests.get(url, params=params).json()
    items = response.get("items", [])
    if items:
        return items[0].get("snippet", "No snippet available.")
    return "No results found."

@tool
def run_python(code: str) -> dict:
    """Execute Python code and return result."""
    try:
        # Create a local dictionary for safe execution
        local_vars = {}
        exec(code,locals=local_vars)
        return local_vars if local_vars else {"Code executed successfully"}
    except Exception as e:
        return f"Error: {e}"


tool_kit = [google_search,run_python]

In [95]:
# Configure LLM
apikey = os.getenv('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite',api_key=apikey)
llm_with_tool = llm.bind_tools(tool_kit)

In [96]:
class State(TypedDict):
    messages : Annotated[list,add_messages]

def llm_node(state:State) -> State:
    return {'messages':[llm_with_tool.invoke(state['messages'])]}

In [97]:
# LangGraph

builder = StateGraph(State)

builder.add_node('LLM_Node',llm_node)
builder.add_node('tools',ToolNode(tool_kit))

builder.add_edge(START,"LLM_Node")
builder.add_conditional_edges('LLM_Node',tools_condition)
builder.add_edge("tools", "LLM_Node")

graph = builder.compile()

In [98]:
print(graph.get_graph().draw_ascii())

        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
         +----------+         
         | LLM_Node |         
         +----------+         
          .         .         
        ..           ..       
       .               .      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 


In [111]:
input_message = [{'role':'user','content':'Use Tool and tell Who invented the television?'}]
response = graph.invoke({'messages':input_message})
ans = response['messages'][-1].content

In [112]:
ans

'Philo Taylor Farnsworth was the inventor of television. He was granted the first patent for a fully electronic television system.'

In [113]:
response

{'messages': [HumanMessage(content='Use Tool and tell Who invented the television?', additional_kwargs={}, response_metadata={}, id='4d864e22-9b74-46ff-bb77-b033ca6a98e7'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'google_search', 'arguments': '{"query": "Who invented the television?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--8c5a7b25-e719-4324-b849-64f287c1df2d-0', tool_calls=[{'name': 'google_search', 'args': {'query': 'Who invented the television?'}, 'id': '1a804738-add6-4275-b2b5-0ee3858c9952', 'type': 'tool_call'}]),
  ToolMessage(content='Philo Taylor Farnsworth (August 19, 1906 – March 11, 1971), "The father of television", was the American inventor and pioneer who was granted the first\xa0...', name='google_search', id='04421d41-e739-4f2c-ada2-4226ebb60415', tool_call_id='1a804738-add6-4275-b2b5-0ee3858c9952'),
 

In [114]:
input_message = [{'role':'user','content':'Use tool and tell what is the date today?'}]
response = graph.invoke({'messages':input_message})
ans = response['messages'][-1].content

In [115]:
ans

'I am sorry, I cannot fulfill this request. The available tools do not have the ability to access or provide the current date.'

In [116]:
response

{'messages': [HumanMessage(content='Use tool and tell what is the date today?', additional_kwargs={}, response_metadata={}, id='5290c479-6036-44ab-89f1-55cc440df9c8'),
  AIMessage(content='I am sorry, I cannot fulfill this request. The available tools do not have the ability to access or provide the current date.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--284bf524-1d78-4349-9c2f-0da3831bd2da-0')]}

In [117]:
input_message = [{'role':'user','content':'What was the maximum temperature of Delhi NCR today? use google_search'}]
response = graph.invoke({'messages':input_message})
ans = response['messages'][-1].content

In [118]:
ans

'The maximum temperature in Delhi NCR today is 28°C.'

In [119]:
response

{'messages': [HumanMessage(content='What was the maximum temperature of Delhi NCR today? use google_search', additional_kwargs={}, response_metadata={}, id='25c57482-3161-4a03-b740-e7c0892455e5'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'google_search', 'arguments': '{"query": "maximum temperature Delhi NCR today"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--296d3483-f8d4-4b5e-a9a0-7f8046439263-0', tool_calls=[{'name': 'google_search', 'args': {'query': 'maximum temperature Delhi NCR today'}, 'id': 'eae69b0b-a2a2-4769-92f1-8e0a25375f34', 'type': 'tool_call'}]),
  ToolMessage(content="Today's Weather in Delhi: In Delhi today, the weather is expected to be Partly Cloudy with a maximum temperature of 28°C and a minimum of 17°C. Sunrise in\xa0...", name='google_search', id='e364348c-0f0b-4b7f-bd51-4c14d8b0a60d', tool_call_id='

In [120]:
input_message = [{'role':'user','content':'add 2 in every element of list a=[7,9,3,5]. use tool'}]
response = graph.invoke({'messages':input_message})
ans = response['messages'][-1].content

[9, 11, 5, 7]


In [121]:
ans

''

In [122]:
response

{'messages': [HumanMessage(content='add 2 in every element of list a=[7,9,3,5]. use tool', additional_kwargs={}, response_metadata={}, id='2f2c3372-2246-42a4-b239-7c65eb2cbfb9'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'run_python', 'arguments': '{"code": "a = [7, 9, 3, 5]\\nnew_a = [x + 2 for x in a]\\nprint(new_a)"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--7baf5eb6-ec39-45ec-92e0-a6919af838eb-0', tool_calls=[{'name': 'run_python', 'args': {'code': 'a = [7, 9, 3, 5]\nnew_a = [x + 2 for x in a]\nprint(new_a)'}, 'id': 'bf9bc924-db0b-4ca5-ae70-9336a017d20b', 'type': 'tool_call'}]),
  ToolMessage(content='{"a": [7, 9, 3, 5], "new_a": [9, 11, 5, 7]}', name='run_python', id='d05befcc-9a4d-4f51-8e0e-8562728aa1e4', tool_call_id='bf9bc924-db0b-4ca5-ae70-9336a017d20b'),
  AIMessage(content='', additional_kwargs={}, response_meta